In [1]:
import torch
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
import numpy as np
# np.argmax([1,2,3,4])
def softmax(x, axis=None):
    x = np.array(x)
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    rst = y / y.sum(axis=axis, keepdims=True)
    return rst.tolist()
softmax([1,2])
tokenizer = AutoTokenizer.from_pretrained('ethanyt/guwenbert-base')

In [2]:
import json
with open('data/test_placeholder.jsonl','r') as f:
    valid_json = f.readlines()
    valid_json = [json.loads(e) for e in valid_json]

In [3]:
def save_jsonl(predict_valid_json,out_path):
    with open(out_path,'w') as f:
        for line in predict_valid_json:
            this_line = {'translation':line['translation'],'choices':line['choices'],'answer':int(line['predict'])}
            f.write(json.dumps(this_line,ensure_ascii=False)+'\n')

### co_num

In [4]:
def multi_argmin(l):
    min_l = min(l)
    return [i for i in range(len(l)) if l[i] == min_l]
# def choice_cooccurrence(choices):
#     num = len(choices)
#     choices_return = []
#     scores = [0] * num
#     choice_tokens = [[] for _ in range(num)]
#     for i, choice in enumerate(choices):
#         choices_return.append(choice)
#         for c in choice:
#             if c in choice_tokens[i]:
#                 continue
#             choice_tokens[i].append(c)
#     for i in range(num):
#         for j in range(num):
#             if i == j:
#                 continue
#             for c in choice_tokens[i]:
#                 if c in choice_tokens[j]:
#                     scores[i] += 1
#     min_choice = multi_argmin(scores)
#     if len(min_choice) > 3:
#         return choices_return
#     for i in min_choice:
#         choices_return[i] = ''
#     return choices_return
# valid_json[0]['choices'],choice_cooccurrence(valid_json[0]['choices'])

In [5]:
def calculate_translation_with_choice(translations, choice):
    rst = 0
    for c in choice:
        if c in translations:
            rst += 1
    return rst
def calculate_choice_in_choices(choice,choices):
    rst = 0
    for c in choice:
        for other_choice in choices:
            if choice == other_choice:
                continue
            if c in other_choice:
                rst += 1
    return rst
def calculate_choice_in_position_choices(choice,choices):
    rst = 0
    for c_id in range(len(choice)):
        c = choice[c_id]
        for other_choice in choices:
            if choice == other_choice:
                continue
            other_c = other_choice[c_id]
            if c == other_c:
                rst += 1
    return rst
    
predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
#     choices = choice_cooccurrence(choices)
    this_scores = []
    for choice_id in range(len(choices)):
        choice_score = calculate_translation_with_choice(valid_json[i]['translation'], choices[choice_id])
        choice_score += 0.5 * calculate_choice_in_choices(choices[choice_id], choices)
#         choice_score += calculate_choice_in_position_choices(choices[choice_id], choices)
#         choice_score = sum(choice_score)
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = softmax(this_scores)
    this_dict['scores'] = this_scores
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['no_predict'] = multi_argmin(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

0.3025735294117647

In [6]:
co_num_predict = predict_valid_json

In [7]:
co_num_predict[-1]

{'translation': '你还在斗鸡场妒敌自鸣得意。',
 'choices': ['禽心亦自妒专场', '诗场且自横驰骋', '妒敌专场好自娱', '竿木逢场且自娱'],
 'answer': 0,
 'scores': [0.16467143424506278,
  0.03674316349497249,
  0.7380061670529912,
  0.06057923520697361],
 'predict': 2,
 'no_predict': [1],
 'true': False}

### ner


In [8]:
import pickle
with open('predict/final/test/ner.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for batch_idx in tqdm(range(len(rst['test_dataset']))):
    origin_id = rst['test_dataset'][batch_idx]['origin_idx']
    if origin_id not in all_scores:
        all_scores[origin_id] = []
    predicts = softmax(rst['test_result'].predictions[batch_idx],axis=-1)
    labels = rst['test_result'].label_ids[batch_idx]
    all_scores[origin_id].append(np.array(predicts)[np.where(labels!=-100)[0].tolist()][:,1].tolist())
def multiple_list(l):
    rst = 1
    for e in l:
        rst *= e
    return rst

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = []
    for choice_id in range(len(choices)):
        choice_score = all_scores[i][choice_id]
        choice_score = multiple_list(choice_score)
#         choice_score = sum(choice_score)
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

100%|████████████████████████████████████████████████████████████████████████| 10880/10880 [00:09<00:00, 1151.62it/s]


In [9]:
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

0.25882352941176473

In [10]:
ner_predict = predict_valid_json

### binary

In [11]:
import pickle
with open('predict/final/test/binary.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = rst['test_dataset'][i]['origin_idx']
    input_str = rst['test_dataset'][i]['input_ids']
    input_str = tokenizer.decode(input_str)
    input_str = input_str.split('[SEP]')
    if idx not in all_scores:
        all_scores[idx] = {}
    all_scores[idx][input_str[1].replace(' ','')] = softmax(rst['test_result'].predictions[i].tolist())

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = []
    for choice in choices:
        choice_score = 0
        for k in all_scores[i]:
            if k in choice:
                choice_score += all_scores[i][k][1]
        this_scores.append(choice_score)
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)

np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

100%|████████████████████████████████████████████████████████████████████████| 24749/24749 [00:09<00:00, 2500.93it/s]


0.2584558823529412

In [12]:
binary_predict = predict_valid_json

### seq

In [13]:
import pickle
with open('predict/final/test/seq.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = i
    all_scores[idx] = softmax(rst['test_result'].predictions[i].tolist())

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = all_scores[i]
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

100%|█████████████████████████████████████████████████████████████████████████| 2720/2720 [00:00<00:00, 40849.86it/s]


0.2610294117647059

In [14]:
seq_predict = predict_valid_json

### baseline

In [15]:
import pickle
with open('predict/final/test/baseline.pkl','rb') as f:
    rst = pickle.load(f)
all_scores = {}
for i in tqdm(range(len(rst['test_dataset']))):
    idx = i
    all_scores[idx] = softmax(rst['test_result'].predictions[i].tolist())

predict_valid_json = []
for i in range(len(valid_json)):
    choices = valid_json[i]['choices']
    this_scores = all_scores[i]
    this_dict = {k:valid_json[i][k] for k in valid_json[i]}
    this_scores = np.array(this_scores)
#     this_scores[co_num_predict[i]['no_predict']] += -100
    this_scores = this_scores.tolist()
    this_dict['scores'] = softmax(this_scores)
    this_dict['predict'] = np.argmax(this_scores)
    this_dict['true'] = (this_dict['predict'] == this_dict['answer'])
    predict_valid_json.append(this_dict)
np.sum([int(e['true']) for e in predict_valid_json]) / len(predict_valid_json)

100%|█████████████████████████████████████████████████████████████████████████| 2720/2720 [00:00<00:00, 39877.20it/s]


0.2613970588235294

In [16]:
baseline_predict = predict_valid_json

### save_each

In [17]:
save_jsonl(ner_predict,'predict/final/test/ner.jsonl')
save_jsonl(binary_predict,'predict/final/test/binary.jsonl')
save_jsonl(seq_predict,'predict/final/test/seq.jsonl')
save_jsonl(baseline_predict,'predict/final/test/baseline.jsonl')

### ensemble

In [35]:
oracle_true = [binary_predict[i]['true'] or ner_predict[i]['true'] or seq_predict[i]['true'] for i in range(len(ner_predict))]
oracle_true = sum(oracle_true) / len(oracle_true)
print(oracle_true)

0.31286764705882353


In [18]:
w = [1,0.1,1.5,2]
predict_list = [ner_predict,binary_predict,seq_predict,co_num_predict]
ensemble_predict = []
def get_score(w,predict_list):
    ensemble_predict = []
    for i in range(len(predict_list[0])):
        ensemble_predict.append(predict_list[0][i].copy())
        ensemble_predict[-1]['scores'] = [np.array(predict_list[each_ensemble][i]['scores']) * w[each_ensemble] 
                                          for each_ensemble in range(len(predict_list))]
        ensemble_predict[-1]['scores'] = sum(ensemble_predict[-1]['scores']).tolist()
        ensemble_predict[-1]['predict'] = np.argmax(ensemble_predict[-1]['scores'])
        ensemble_predict[-1]['true'] = (ensemble_predict[-1]['predict'] == ensemble_predict[-1]['answer'])
    return np.sum([int(e['true']) for e in ensemble_predict]) / len(ensemble_predict),ensemble_predict
rst,ensemble_predict = get_score(w,predict_list)
print(rst)

0.26176470588235295


In [19]:
save_jsonl(ensemble_predict,'predict/final/test/ensemble.jsonl')

In [21]:
for i in range(100):
    w1 = i * 0.1
    print(w1,' ',get_score([w[0],w[1],w1],predict_list))

0.0   (0.26286764705882354, [{'translation': '侍奉夫主，不能尽自己的天年。', 'choices': ['事主不尽年', '事君期尽年', '事主不及终', '万年不尽意'], 'answer': 0, 'scores': [0.6052434050383504, 0.27315615065284266, 2.1840579938378034e-44, 0.22160044430880713], 'predict': 0, 'true': True}, {'translation': '触痛了山南蜀客的似箭归心。', 'choices': ['前山蜀客归', '前溪有归客', '蜀客亦思归', '青山待客归'], 'answer': 0, 'scores': [0.41960175532785593, 1.238114866207428e-44, 0.34739296900994704, 0.33300527566219706], 'predict': 0, 'true': True}, {'translation': '为王事奔波路程尚没有走尽。', 'choices': ['王程应未尽', '王程尚未安', '王程余未尽', '王程未可稽'], 'answer': 0, 'scores': [0.2839007153790332, 0.2724135296306003, 0.5436857549903665, 1.0434616584968068e-44], 'predict': 2, 'true': False}, {'translation': '可惜到了黄昏，娇艳芳香的花儿凋零。', 'choices': ['日暮槿花零落风', '可怜日暮嫣香落', '可怜日落长安路', '可怜日暮蔫香落'], 'answer': 0, 'scores': [6.387677179741781e-45, 0.46663281180525784, 0.18425081980277397, 0.4491163683919682], 'predict': 1, 'true': False}, {'translation': '是你又狂放又侠义又温文尔雅。', 'choices': ['亦狂亦侠亦温文', '狂侠何妨亦粹温', '不饮

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

